In [5]:
import numpy
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Flatten
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.utils import np_utils
from keras import backend as K
K.set_image_dim_ordering('th')
from sklearn.model_selection import train_test_split
# fix random seed for reproducibility
seed = 7
numpy.random.seed(seed)

In [6]:
(X_train, y_train), (X_test, y_test) = mnist.load_data()
X_val, X_test, y_val, y_test = train_test_split(X_test, y_test, test_size=0.33, random_state=seed)
# reshape to be [samples][pixels][width][height]
X_train = X_train.reshape(X_train.shape[0], 1, 28, 28).astype('float32')
X_val = X_val.reshape(X_val.shape[0], 1, 28, 28).astype('float32')
X_test = X_test.reshape(X_test.shape[0], 1, 28, 28).astype('float32')

In [7]:
# normalize inputs from 0-255 to 0-1
X_train = X_train / 255
X_val = X_val / 255
X_test = X_test / 255
# one hot encode outputs
y_train = np_utils.to_categorical(y_train)
y_val = np_utils.to_categorical(y_val)
y_test = np_utils.to_categorical(y_test)
num_classes = y_val.shape[1]

In [8]:
def baseline_model():
	# create model
	model = Sequential()
	model.add(Conv2D(32, (5, 5), input_shape=(1, 28, 28), activation='relu'))
	model.add(MaxPooling2D(pool_size=(2, 2)))
	model.add(Dropout(0.2))
	model.add(Flatten())
	model.add(Dense(128, activation='relu'))
	model.add(Dense(num_classes, activation='softmax'))
	# Compile model
	model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
	return model

In [9]:
# build the model
model = baseline_model()
# Fit the model
model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=10, batch_size=200, verbose=2)
# Final evaluation of the model
scores = model.evaluate(X_test, y_test, verbose=0)
print("CNN Error: %.2f%%" % (100-scores[1]*100))

Train on 60000 samples, validate on 6700 samples
Epoch 1/10
 - 219s - loss: 0.2235 - acc: 0.9365 - val_loss: 0.0797 - val_acc: 0.9742
Epoch 2/10
 - 210s - loss: 0.0716 - acc: 0.9786 - val_loss: 0.0428 - val_acc: 0.9852
Epoch 3/10
 - 211s - loss: 0.0512 - acc: 0.9842 - val_loss: 0.0433 - val_acc: 0.9846
Epoch 4/10
 - 161s - loss: 0.0393 - acc: 0.9879 - val_loss: 0.0403 - val_acc: 0.9878
Epoch 5/10
 - 188s - loss: 0.0328 - acc: 0.9894 - val_loss: 0.0333 - val_acc: 0.9890
Epoch 6/10
 - 170s - loss: 0.0266 - acc: 0.9916 - val_loss: 0.0322 - val_acc: 0.9900
Epoch 7/10
 - 179s - loss: 0.0225 - acc: 0.9928 - val_loss: 0.0337 - val_acc: 0.9888
Epoch 8/10
 - 180s - loss: 0.0189 - acc: 0.9940 - val_loss: 0.0334 - val_acc: 0.9884
Epoch 9/10
 - 164s - loss: 0.0155 - acc: 0.9949 - val_loss: 0.0314 - val_acc: 0.9904
Epoch 10/10
 - 156s - loss: 0.0142 - acc: 0.9956 - val_loss: 0.0315 - val_acc: 0.9890
CNN Error: 1.03%


In [11]:
import matplotlib.pyplot as plt
f, (ax1, ax2) = plt.subplots(1,2, figsize = (20,10))
index = 10
ax1.imshow(X_test[index].reshape(28,28), cmap = 'gray')
ax2.bar([0,1,2,3,4,5,6,7,8,9], pred[index])
plt.show()

NameError: name 'pred' is not defined

In [ ]:
w = model.layers[0].get_weights()[0]

In [ ]:
f, ax = plt.subplots(4,8, figsize = (80,40))
for i in range(4):
    for j in range(8):
        ax[i][j].imshow(w[:,:,0,i*8+j], cmap = 'gray')
plt.show()

In [ ]:
# visualizing feature maps of the CNN
from keras import models
layer_outputs = [layer.output for layer in model.layers[:12]] 
# Extracts the outputs of the top 12 layers
activation_model = models.Model(inputs=model.input, outputs=layer_outputs) # Creates a model that will return these outputs, given the model input

In [ ]:
activations = activation_model.predict(X_test) 
# Returns a list of five Numpy arrays: one array per layer activation

In [ ]:
first_layer_activation = activations[0]
print(first_layer_activation.shape)

In [ ]:
ind = 30
plt.imshow(X_test[ind].reshape(28,28), cmap = 'gray')
plt.show()
f, ax = plt.subplots(4,8, figsize = (80,40))
for i in range(4):
    for j in range(8):
        ax[i][j].imshow(first_layer_activation[ind, i, :, :], cmap = 'gray')
plt.show()